In [2]:
import pandas as pd
import sqlite3
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline


In [4]:
def run_query(database_path, query):
    # Connect to the SQLite database
    # If the database does not exist, it will be created
    connection = sqlite3.connect(database_path)
    
    try:
        # Create a cursor object using the connection
        cursor = connection.cursor()
        
        # Execute the SQL query
        cursor.execute(query)
        
        # Fetch all results from the executed query
        results = cursor.fetchall()
        
        # Turning list of tuples to list of strings for further processing
        return [str(t[0]) for t in results]

            
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
        
    finally:
        
        # Close the connection to the database
        if connection:
            connection.close()


In [9]:
database_path = '../openregister.db'  # Path to your SQLite database file
query = 'SELECT name FROM company LIMIT 250;'  # Replace 'your_table' with your actual table name

company_names = run_query(database_path, query)


In [ ]:

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

results = []
for company_name in company_names:
    results.append(nlp(company_name))
# Flatten the list of lists into a single list
flat_data = [item for sublist in results for item in sublist]

# Convert to DataFrame
df = pd.DataFrame(flat_data)
print(df)

/Users/dominikpichler/Documents/Git/OpenSource/Balmung/ER-NLP/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactl